A **kernel** lets you solve nonlinear problems using methods that were originally designed for linear models, by working with a similarity function between points instead of explicit coordinates in a feature space. 

The idea is developed first for regularized linear regression, then generalized toward the “kernel trick” used in SVMs and other algorithms.

### 1. Intuitive picture: why kernels?

#### 1.1 Linear vs nonlinear separation

- A linear model (like plain linear regression or logistic regression) uses a prediction of the form  
  $\hat{y}(x) = \beta^\top \phi(x)$,  
  where $\phi(x)$ is a vector of features (maybe just the original inputs, maybe some nonlinear transforms).
- In a 2D plot, such a model produces **straight** decision boundaries (lines).  
- Many real datasets are **not** linearly separable in the original space: you might need a curve or complex boundary.

A common trick is to **map the data into a higher-dimensional feature space** using nonlinear features—e.g., quadratics, cubics, trigonometric functions—where a straight boundary becomes possible.

Example (conceptual):

- In 2D, points of one class form a ring around the origin; the other class is in the center.
- No straight line can separate them in 2D.
- If you add the feature $r^2 = x_1^2 + x_2^2$, in the new space $(x_1, x_2, r^2)$ you can separate them using a hyperplane.

Kernels formalize and generalize this idea.

#### 1.2 What is a kernel?

A **kernel** is a function $k(x, x')$ that takes two data vectors and returns a single number measuring their **similarity** in some (possibly high-dimensional) feature space.

Key points:

- You can think of a kernel as computing  
  $k(x, x') = \phi(x)^\top \phi(x')$  
  for some feature map $\phi$, even if you never explicitly compute $\phi(x)$.
- Different kernels correspond to different choices of $\phi$ and, therefore, different kinds of nonlinear structure the model can capture.

Common kernels (for SVM, etc.):

- Linear: $k(x, x') = x^\top x'$ (no extra features)
- Polynomial: $k(x, x') = (\gamma x^\top x' + r)^d$
- RBF/Gaussian: $k(x, x') = \exp(-\gamma \|x - x'\|^2)$

The **kernel trick**: if an algorithm can be written purely in terms of inner products $x_i^\top x_j$, you can replace those inner products with a kernel $k(x_i, x_j)$. That implicitly moves you to a high-dimensional feature space without ever computing coordinates there.

### 2. Linear regression with nonlinear features (primal view)

Start with **regularized linear regression** with nonlinear features, because the math is clean and the structure generalizes to other algorithms.

#### 2.1 Model and notation

We have data:

- Inputs: $x_1, \dots, x_N$
- Outputs: $y_1, \dots, y_N$

We define a feature map $\phi(x)$ (vector of $M$ features):

- The 0th feature is always $1$ (bias).
- The remaining $M - 1$ features are nonlinear transforms of $x$ (e.g., polynomials, etc.).

We collect:

- Parameter vector $\beta \in \mathbb{R}^M$: $\beta = (\beta_0, \dots, \beta_{M-1})^\top$
- Feature vector for sample $i$: $\phi(x_i) \in \mathbb{R}^M$

The model is:

$$
\hat{y}(x_i) = \beta^\top \phi(x_i).
$$

#### 2.2 Regularized least squares loss

We want to find $\beta$ that minimizes:

$$
\mathcal{L}(\beta)
= \sum_{i=1}^N (\beta^\top \phi(x_i) - y_i)^2
+ \frac{\lambda}{2} \|\beta\|^2,
$$

where $\lambda > 0$ is a regularization strength:

- First term: sum of squared errors.
- Second term: L2 penalty (ridge) on coefficients, discouraging very large $\beta$.

The factor $\frac{\lambda}{2}$ is just for algebraic convenience.

#### 2.3 Matrix form (primal / β-formulation)

Define:

- Feature matrix $\Phi \in \mathbb{R}^{N \times M}$: each row is $\phi(x_i)^\top$.
- Target vector $Y \in \mathbb{R}^N$: entries $y_i$.

Then:

$$
\mathcal{L}(\beta)
= \|\Phi \beta - Y\|^2 + \frac{\lambda}{2} \|\beta\|^2.
$$

This is a convex quadratic in $\beta$. The minimizer is found by setting the derivative to zero.

Compute gradient and set to zero:

$$
\frac{\partial \mathcal{L}}{\partial \beta}
= 2\Phi^\top (\Phi\beta - Y) + \lambda \beta = 0.
$$

Rearrange:

$$
(\Phi^\top \Phi + \lambda I)\beta = \Phi^\top Y.
$$

So the **closed-form solution** (primal solution) is:

$$
\beta^\ = (\Phi^\top \Phi + \lambda I)^{-1} \Phi^\top Y.
$$

Here:

- $\Phi^\top \Phi$ is an $M \times M$ matrix.
- You invert an $M \times M$ matrix: complexity ~ $O(M^3)$.

Prediction for a new point $x_{\text{new}}$:

$$
\hat{y}(x_{\text{new}}) = \beta^{*\top} \phi(x_{\text{new}}).
$$

Notice:

- After computing $\beta^{*\top}$, you can forget the training data and just keep $\beta^{*\top}$.
- This is the **primal**, or **β-based**, viewpoint.

### 3. Alternative formulation: α-parameters (dual-like view)

An alternative set of parameters $\alpha_i$, one per data point, to expose where kernels will appear.

#### 3.1 Define α in terms of errors

Define:

$$
\alpha_i = -\frac{\beta^\top \phi(x_i) - y_i}{\lambda}.
$$

Equivalently:

$$
\alpha_i = \frac{1}{\lambda} (y_i - \beta^\top \phi(x_i)).
$$

So $\alpha_i$ is (up to scaling) the negative prediction error for point $i$.

We can write:

- Vector $\alpha \in \mathbb{R}^N$: entries $\alpha_i$.

From the gradient condition:

$$
2\Phi^\top (\Phi\beta - Y) + \lambda \beta = 0
$$

Divide by 2 for simplicity:

$$
\Phi^\top (\Phi\beta - Y) + \frac{\lambda}{2}\beta = 0
$$

But it’s simpler to view the derivation in the way presented in the transcript:

Using the definition of $\alpha_i$ and some algebra, you can show:

$$
\beta = \Phi^\top \alpha.
$$

This says: **β is a linear combination of the feature vectors of the training points**.

#### 3.2 Matrix equations for α

We also plug the definition of $\alpha$ into matrix form:

$$
- \lambda \alpha = \Phi \beta - Y.
$$

Call that equation (4) in the transcript. Combine:

- $\beta = \Phi^\top \alpha$
- $-\lambda \alpha = \Phi\beta - Y$

Substitute $\beta = \Phi^\top \alpha$ into the second:

$$
- \lambda \alpha = \Phi (\Phi^\top \alpha) - Y = (\Phi\Phi^\top)\alpha - Y.
$$

So:

$$
(\Phi\Phi^\top + \lambda I)\alpha = Y.
$$

Therefore, the **α-solution** is:

$$
\alpha^* = (\Phi\Phi^\top + \lambda I)^{-1} Y.
$$

Now:

- $\Phi\Phi^\top$ is an $N \times N$ matrix.
- You invert an $N \times N$ matrix: complexity ~ $O(N^3)$.

This looks **worse** than the β-formulation if $N \gg M$ (many more data points than features), which is typical.

#### 3.3 Predictions in terms of α

We still know:

$$
\beta^* = \Phi^\top \alpha^*.
$$

So for a new point $x_{\text{new}}$:

$$
\hat{y}(x_{\text{new}}) 
= \beta^{*\top} \phi(x_{\text{new}})
= \alpha^{*\top} \Phi \phi(x_{\text{new}}).
$$

Expand $\Phi \phi(x_{\text{new}})$:

- The $i$-th entry is $\phi(x_i)^\top \phi(x_{\text{new}})$.

So:

$$
\hat{y}(x_{\text{new}}) = \sum_{i=1}^N \alpha_i^* \phi(x_i)^\top \phi(x_{\text{new}}).
$$

Key observation:

- With β-formulation, predictions only need $\beta$.
- With α-formulation, predictions require **all training points** (through the sums).
- Even without kernels, this seems computationally and memory-wise worse.

So why is this useful? Because of what $\phi(x_i)^\top \phi(x_j)$ really is.

### 4. The kernel matrix and kernel function

Look at $\Phi\Phi^\top$:

- Entry $(i, j)$ is $\phi(x_i)^\top \phi(x_j)$.
- This is the **dot product** of feature vectors for points $i$ and $j$.
- It measures **similarity** between the two data points **in feature space**.

We call:

- $K_{ij} = \phi(x_i)^\top \phi(x_j)$  
  the **kernel matrix** $K$.
- The function  
  $k(x, x') = \phi(x)^\top \phi(x')$  
  the **kernel function**.

In the α-formulation:

- Training uses $\Phi\Phi^\top$, i.e., **only dot products** of the form $\phi(x_i)^\top \phi(x_j)$.
- Prediction uses sums of $\phi(x_i)^\top \phi(x_{\text{new}})$, again **only dot products**.

This reveals the main insight:

> If an algorithm (in this alternative form) only needs dot products of feature vectors $\phi(x)$, then you never need to know $\phi(x)$ explicitly—only a function that gives you these dot products.

That function is the kernel $k(x, x')$.

So we can:

- **Replace** $\phi(x_i)^\top \phi(x_j)$ with **any** valid kernel $k(x_i, x_j)$.
- This lets us work in a **potentially infinite-dimensional feature space** implicitly.
- We avoid ever computing explicit coordinates in that space.

This is the **kernel trick**.

### 5. Why the kernel trick matters

#### 5.1 Computation and representation

Instead of:

- Choosing a finite set of nonlinear features and building $\phi(x)$ explicitly.
- Constructing a potentially huge $\Phi$ (with possibly thousands or millions of features).
- Inverting an $M \times M$ matrix.

We can:

- Choose a kernel $k(x, x')$ that corresponds to an **implicit** feature map.
- Work directly with the kernel matrix $K$ of size $N \times N$.
- Solve for α and make predictions using only kernel evaluations.

Even if the implicit feature space is very high-dimensional (or infinite-dimensional, as with the RBF kernel), computations are done in terms of $N$ and kernel calls, not $M$.

#### 5.2 Applicability to many algorithms

Kernels are not just for SVMs. Any algorithm that can be written in terms of inner products can be “kernelized,” including:

- Linear regression (as shown).
- Logistic regression.
- Principal Component Analysis (PCA) → kernel PCA.
- Others that rely on distances/inner products.

The general pattern:

1. Rewrite the algorithm in terms of dot products $\phi(x_i)^\top \phi(x_j)$.
2. Replace those with $k(x_i, x_j)$.
3. Choose a kernel appropriate to the problem.

### 6. Simple Python implementations (from basics)

Below are simple examples (for intuition) showing:

- Primal ridge regression (β-form).
- Kernel ridge regression (α-form) using a linear kernel.
- How to generalize to, e.g., an RBF kernel.


#### 6.1 Primal ridge regression with nonlinear features

We’ll illustrate in 1D for clarity: fitting a nonlinear curve with explicit polynomial features.

Here:

- We chose the features explicitly (up to degree 3).
- We solved for $\beta$ via $(\Phi^\top \Phi + \lambda I)^{-1} \Phi^\top Y$.
- All computations are in the **feature space** via $\Phi$.

In [1]:
import numpy as np
from numpy.linalg import inv

# Generate toy data
np.random.seed(0)
N = 30
X = np.linspace(-3, 3, N).reshape(-1, 1)
y = np.sin(X).ravel() + 0.1 * np.random.randn(N)

# Build feature matrix Phi with polynomial features [1, x, x^2, x^3]
def poly_features(x, degree=3):
    # x is shape (N, 1)
    N = x.shape[0]
    Phi = np.ones((N, degree + 1))  # columns: x^0, x^1, ..., x^degree
    for d in range(1, degree + 1):
        Phi[:, d] = x[:, 0] ** d
    return Phi

Phi = poly_features(X, degree=3)
Y = y.reshape(-1, 1)

lam = 0.1  # regularization strength

# Closed-form ridge solution in primal
M = Phi.shape[1]
beta = inv(Phi.T @ Phi + lam * np.eye(M)) @ Phi.T @ Y  # shape (M, 1)

# Prediction for new x values
X_new = np.linspace(-3, 3, 100).reshape(-1, 1)
Phi_new = poly_features(X_new, degree=3)
y_pred = Phi_new @ beta


#### 6.2 Kernel ridge regression: linear kernel (as a sanity check)

Now we implement the α-form using a kernel.

For a **linear kernel** and certain choices of features, the primal and dual give essentially the same function class, but this already shows:

- Training involves inverting an $N \times N$ matrix.
- Prediction involves sums of kernel evaluations against all training points.

In [2]:
def linear_kernel(X1, X2):
    # X1: (N, d), X2: (M, d)
    return X1 @ X2.T

# Kernel matrix K = Phi Phi^T, but here we use X directly
K = linear_kernel(X, X)  # shape (N, N)

# Solve for alpha: (K + lam * I) alpha = y
alpha = np.linalg.inv(K + lam * np.eye(N)) @ y.reshape(-1, 1)  # shape (N, 1)

# Predict for new points
K_new = linear_kernel(X_new, X)  # (N_new, N)
y_pred_kernel = K_new @ alpha  # (N_new, 1)

#### 6.3 Kernel ridge regression with RBF kernel

Now replace the linear kernel with a more powerful one, e.g., RBF.

Notes:

- We never constructed explicit high-dimensional features $\phi(x)$.
- The RBF kernel implicitly corresponds to an **infinite-dimensional** feature space.
- Yet training and prediction both use only kernel evaluations.

This is a concrete example of the **kernel trick** in action.

In [3]:
def rbf_kernel(X1, X2, gamma=1.0):
    # squared Euclidean distance (broadcasting)
    # X1: (N, d), X2: (M, d)
    X1_sq = np.sum(X1**2, axis=1).reshape(-1, 1)  # (N, 1)
    X2_sq = np.sum(X2**2, axis=1).reshape(1, -1)  # (1, M)
    sq_dists = X1_sq + X2_sq - 2 * X1 @ X2.T      # (N, M)
    return np.exp(-gamma * sq_dists)

gamma = 0.5
K = rbf_kernel(X, X, gamma=gamma)
alpha = np.linalg.inv(K + lam * np.eye(N)) @ y.reshape(-1, 1)

K_new = rbf_kernel(X_new, X, gamma=gamma)
y_pred_rbf = K_new @ alpha

### 7. Connecting to SVMs and beyond

In SVMs (which the module will focus on next):

- The optimization problem can be written in “dual” form using only dot products of inputs.
- Replacing dot products with kernels turns a **linear SVM** into a **nonlinear SVM**.
- The resulting decision boundary in the original space can be dramatically nonlinear, while the algorithm itself uses only a kernel function and constraints on α-like coefficients.

Similarly, kernel versions exist for:

- Logistic regression (kernel logistic regression).
- PCA (kernel PCA).
- Other algorithms that rely fundamentally on inner products.

The big picture:

- Many linear algorithms **depend only on geometry** (relative positions, angles, distances).
- Geometry can often be captured fully by inner products.
- Kernels let us **redefine geometry** in a richer feature space without ever working in that space explicitly.